This script loads all of the hourly data.  It also creates daily data by aggregating the hourly data. It then saves the created data structures as pickles (compressed binary files) for faster loading in the future.

# Forecasting Data #

In [1]:
# imports
import pandas as pd
import numpy as np
import time
from timeit import default_timer as timer
import gc

In [2]:
# define data location
#location = '/Users/mithras/Documents/_SCHOOL/_Drexel/BUSN 710 - Capstone/Data/Forecasting Project/'
location = '/Users/loki/Documents/_SCHOOL/_Drexel/BUSN 710 - Capstone/Data/Forecasting Project/'

## Data Cleaning & Checking 

In [74]:
# # hourly data
# start = timer()

# use_oct_in = pd.read_excel(location+'PECO Zip HourlyUsage_2017.10.xlsx')
# use_nov_in = pd.read_excel(location+'PECO Zip HourlyUsage_2017.11.xlsx') 
# use_dec_in = pd.read_excel(location+'PECO Zip HourlyUsage_2017.12.xlsx') 
# use_jan_in = pd.read_excel(location+'Zip_HourlylUsage_2018.01.xlsx')
# use_feb_in = pd.read_excel(location+'PECO Zip HourlyUsage_2018.02.xlsx')
# use_mar_in = pd.read_excel(location+'PECO Zip HourlyUsage_2018.03.xlsx')
# use_apr_in = pd.read_excel(location+'PECO Zip HourlyUsage_2018.04.xlsx')
# use_may_in = pd.read_excel(location+'PECO Zip HourlyUsage_2018.05.xlsx')
# use_jun_in = pd.read_excel(location+'PECO Zip HourlyUsage_2018.06.xlsx')
# use_jul_in = pd.read_excel(location+'Zip_HourlylUsage_2018.07.xlsx')
# use_aug_in = pd.read_excel(location+'PECO Zip HourlyUsage_2018.08.xlsx')
# use_sep_in = pd.read_excel(location+'PECO Zip HourlyUsage_2018.09.xlsx')

# use_oct_in2 = pd.read_excel(location+'PECO Zip HourlyUsage_2016.10.xlsx')
# use_nov_in2 = pd.read_excel(location+'PECO Zip HourlyUsage_2016.11.xlsx') 
# use_dec_in2 = pd.read_excel(location+'PECO Zip HourlyUsage_2016.12.xlsx') 
# use_jan_in2 = pd.read_excel(location+'PECO Zip HourlyUsage_2017.01.xlsx')
# use_feb_in2 = pd.read_excel(location+'PECO Zip HourlyUsage_2017.02.xlsx')
# use_mar_in2 = pd.read_excel(location+'PECO Zip HourlyUsage_2017.03.xlsx')
# use_apr_in2 = pd.read_excel(location+'PECO Zip HourlyUsage_2017.04.xlsx')
# use_may_in2 = pd.read_excel(location+'PECO Zip HourlyUsage_2017.05.xlsx')
# use_jun_in2 = pd.read_excel(location+'PECO Zip HourlyUsage_2017.06.xlsx')
# use_jul_in2 = pd.read_excel(location+'PECO Zip HourlyUsage_2017.07.xlsx')
# use_aug_in2 = pd.read_excel(location+'PECO Zip HourlyUsage_2017.08.xlsx')
# use_sep_in2 = pd.read_excel(location+'PECO Zip HourlyUsage_2017.09.xlsx')

# end = timer()
# print((end - start)/60) # Time in minutes (~25 min)

In [75]:
# # merge use data
# hourly_in = [use_oct_in, use_nov_in, use_dec_in, use_jan_in, use_feb_in, use_mar_in, 
#              use_apr_in, use_may_in, use_jun_in, use_jul_in, use_aug_in, use_sep_in,
#              use_oct_in2, use_nov_in2, use_dec_in2, use_jan_in2, use_feb_in2, use_mar_in2, 
#              use_apr_in2, use_may_in2, use_jun_in2, use_jul_in2, use_aug_in2, use_sep_in2]
             
# use = pd.concat(hourly_in)

In [76]:
# del (use_oct_in, use_nov_in, use_dec_in, use_jan_in, use_feb_in, use_mar_in, 
#      use_apr_in, use_may_in, use_jun_in, use_jul_in, use_aug_in, use_sep_in,
#      use_oct_in2, use_nov_in2, use_dec_in2, use_jan_in2, use_feb_in2, use_mar_in2, 
#      use_apr_in2, use_may_in2, use_jun_in2, use_jul_in2, use_aug_in2, use_sep_in2)
# del hourly_in

In [77]:
# # save as pickle so we don't have to do this again
# use.to_pickle(location+'fcast_use_raw.pkl.zip')
# del use

In [78]:
# load pickle
start = timer()

use = pd.read_pickle(location+'fcast_use_raw.pkl.zip')

end = timer()
print((end - start)/60) # Time in minutes (~.5 min - MUCH FASTER!!)

0.524653041816661


In [79]:
# create ID tuple
ids = pd.Series(list(map(tuple, use[['DACCOUNTID', 'DMETERNO']].values)))
use['ID'] = ids.values

# find gas records
use = use.loc[use['UOM'] == 'CCF']
use = use.drop(columns=['UOM'])

# convert to datetime
use['Dt'] =  pd.to_datetime(use['METERREADDATE'])

In [80]:
# load customer data
customer_in = pd.read_excel(location+'PECO Zip Customer 2018.10.01 v2.xlsx', sheet_name="Account")
ids = pd.Series(list(map(tuple, customer_in[['DACCOUNTID', 'DMETERNO']].values)))
customer_in['ID'] = ids.values
customer = customer_in.drop(columns=['CITY', 'STATE', 'ZIPCODE', 'COUNTYCODE'])
del customer_in

## Data Restructuring ##

In [81]:
# functions for naming consistency
def decrement(x, startswith, split):
    """
    decrements a passed string of form "demo#" by 1
    
    Parameters
    ----------
    x : string to be decremented
    split : string to split on

    Returns
    ----------
    y : decremented string
    """
    if x.startswith(startswith):
        a,b = x.split(split)
        b = int(b)-1
        y = a + split + str(b)

        return y

    else:
        return x

    
def interval_to_hour(df):
    """
    function for fast rename/relabel during tidying process
    
    Parameters
    ----------
    df : pandas data frame

    Returns
    ----------
    df : data frame with updated column names
    """
    
    df = df.rename(columns=lambda x: decrement(x, "INTERVAL_", "_"))
    df = df.rename(columns=lambda x: x.replace("INTERVAL_", "HR"))
    return df

In [82]:
# rename for consistency
use = interval_to_hour(use)
use = use.drop(columns=['METERREADDATE','HR24'])

In [83]:
# Tidy / Stack data (transform into tall data - one row per customer per hour):
# ref: http://www.jeannicholashould.com/tidy-data-in-python.html
tidy_use = pd.melt(use, 
                   id_vars=['ID','DACCOUNTID','DMETERNO','Dt'],
                   var_name='Hour', value_name='Use')


In [84]:
# drop use from memory
del use

In [85]:
# relabel/retype
tidy_use['Hour'] = tidy_use['Hour'].str.extract('(\d+)').astype(int)
tidy_use = tidy_use.sort_values(by=["Dt"])

### Make daily data ###

In [86]:
# for daily aggregation
daily_use = tidy_use[['ID','Use','Dt']]
daily_use['Dt'] = daily_use['Dt'].dt.date
daily_use['Dt'] =  pd.to_datetime(daily_use['Dt'])
daily_use = daily_use.groupby(['ID','Dt']).sum().reset_index()

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [87]:
# append rate code data
daily_use = daily_use.merge(customer, how='inner', on=['ID'])
#daily_use.head()

In [88]:
# Add dummy variables for day-of-week
daily_use['Weekday'] = pd.get_dummies(daily_use['Dt'].dt.weekday < 5)[True]

# Add dummy variables for holidays
from pandas.tseries.holiday import USFederalHolidayCalendar
cal = USFederalHolidayCalendar()
#dr = pd.date_range(start=min(tidy_use['Dt']), end=max(tidy_use['Dt']))
dr = pd.date_range(start="1-1-2010", end="12-31-2020")
holidays = cal.holidays(start=dr.min(), end=dr.max())

daily_use['Holiday'] = pd.get_dummies(daily_use['Dt'].isin(holidays))[True]
del (USFederalHolidayCalendar, cal, dr, holidays)
#daily_use.head()

In [89]:
# sort by date & time
daily_use = daily_use.sort_values(by=['Dt'])
print(daily_use.shape)
daily_use.head()

(3741870, 11)


,ID,Dt,Use,DACCOUNTID,DCUSTOMERID,DMETERNO,TARIFF,FUELTYPE,REVENUCODE,Weekday,Holiday
1595537,"(152969759120072, 5472501198444)",2016-10-01,243.39,152969759120072,1599795980884,5472501198444,TFG,GAS,5.0,0,0
1595538,"(152969759120072, 5472501198444)",2016-10-02,223.09,152969759120072,1599795980884,5472501198444,TFG,GAS,5.0,0,0
1595539,"(152969759120072, 5472501198444)",2016-10-03,174.88,152969759120072,1599795980884,5472501198444,TFG,GAS,5.0,1,0
1595540,"(152969759120072, 5472501198444)",2016-10-04,214.29,152969759120072,1599795980884,5472501198444,TFG,GAS,5.0,1,0
1595541,"(152969759120072, 5472501198444)",2016-10-05,223.77,152969759120072,1599795980884,5472501198444,TFG,GAS,5.0,1,0


In [90]:
# save file(s) as pickles
# using save location to save with other data files outside of git repo
# data.to_csv(location+'peco.csv', sep='\t')
daily_use.to_pickle(location+'fcast_daily.pkl.zip')
ids.to_pickle(location+'fcast_daily_ids.pkl.zip')
del (daily_use)

### Make hourly data###

In [91]:
# append rate code data
tidy_use = tidy_use.merge(customer, how='inner', on=['ID','DACCOUNTID','DMETERNO'])
#tidy_use.head()

In [92]:
# create single column for date and time
tidy_use['year'] = pd.to_datetime(tidy_use['Dt'].values).year
tidy_use['month'] = pd.to_datetime(tidy_use['Dt'].values).month
tidy_use['day'] = pd.to_datetime(tidy_use['Dt'].values).day
tidy_use['datetime'] = pd.to_datetime(tidy_use[['year','month','day','Hour']])

In [93]:
# Add dummy variables for day-of-week
#use = use.join(pd.get_dummies(use['Dt'].dt.weekday_name))
tidy_use['Weekday'] = pd.get_dummies(tidy_use['Dt'].dt.weekday < 5)[True]

In [94]:
# Add dummy variables for hour-of-day
tidy_use = tidy_use.join(pd.get_dummies(tidy_use['Hour'],prefix='HR'))

# Add dummy variables for part-of-day
tidy_use['MORN'] = pd.get_dummies((tidy_use['Hour'] >= 5) & (tidy_use['Hour'] < 9))[True] # 5-9 am
tidy_use['DAY'] = pd.get_dummies((tidy_use['Hour'] >= 9) & (tidy_use['Hour'] < 17))[True] # 9 am-5 pm
tidy_use['EVE'] = pd.get_dummies((tidy_use['Hour'] >= 17) & (tidy_use['Hour'] < 21))[True] # 5-10 pm
tidy_use['NIGHT'] = pd.get_dummies((tidy_use['Hour'] < 5) | (tidy_use['Hour'] >= 21) )[True] # 10 pm-6 am
# tidy_use.head()

In [95]:
# Add dummy variables for holidays
from pandas.tseries.holiday import USFederalHolidayCalendar
cal = USFederalHolidayCalendar()
#dr = pd.date_range(start=min(tidy_use['Dt']), end=max(tidy_use['Dt']))
dr = pd.date_range(start="1-1-2010", end="12-31-2020")
holidays = cal.holidays(start=dr.min(), end=dr.max())

tidy_use['Holiday'] = pd.get_dummies(tidy_use['Dt'].isin(holidays))[True]
del (USFederalHolidayCalendar, cal, dr, holidays)
#tidy_use.head()

In [96]:
# sort by date & time
tidy_use = tidy_use.drop(columns=['Dt','year','month','day','Hour'])
tidy_use = tidy_use.sort_values(by=['datetime'])
tidy_use = tidy_use.rename(columns={'datetime':'Dt'}) # for consistency
print(tidy_use.shape)
tidy_use.head()

(89804880, 39)


,ID,DACCOUNTID,DMETERNO,Use,DCUSTOMERID,TARIFF,FUELTYPE,REVENUCODE,Dt,Weekday,...,HR_19,HR_20,HR_21,HR_22,HR_23,MORN,DAY,EVE,NIGHT,Holiday
10,"(152969759120072, 5472501198444)",152969759120072,5472501198444,10.60,1599795980884,TFG,GAS,5.0,2016-10-01 00:00:00,0,...,0,0,0,0,0,0,0,0,1,0
15,"(152969759120072, 5472501198444)",152969759120072,5472501198444,9.69,1599795980884,TFG,GAS,5.0,2016-10-01 01:00:00,0,...,0,0,0,0,0,0,0,0,1,0
19,"(152969759120072, 5472501198444)",152969759120072,5472501198444,8.44,1599795980884,TFG,GAS,5.0,2016-10-01 02:00:00,0,...,0,0,0,0,0,0,0,0,1,0
8,"(152969759120072, 5472501198444)",152969759120072,5472501198444,5.81,1599795980884,TFG,GAS,5.0,2016-10-01 03:00:00,0,...,0,0,0,0,0,0,0,0,1,0
17,"(152969759120072, 5472501198444)",152969759120072,5472501198444,4.22,1599795980884,TFG,GAS,5.0,2016-10-01 04:00:00,0,...,0,0,0,0,0,0,0,0,1,0


In [97]:
# save file(s) as pickles
# using save location to save with other data files outside of git repo
# data.to_csv(location+'peco.csv', sep='\t')
tidy_use.to_pickle(location+'fcast_hourly.pkl.zip')
ids.to_pickle(location+'fcast_hourly_ids.pkl.zip')
del (tidy_use,ids)

 ### Merge use & weather? ###

In [98]:
# # load data
# hourly = pd.read_pickle(location+'peco_hourly.pkl.zip')
# sufficient_hourly = pd.read_pickle(location+'peco_sufficient_hourly.pkl.zip')
# weather = pd.read_pickle(location+'hourly_weather.pkl.zip')

In [99]:
# # merge
# hourly = pd.merge(hourly, weather, how='inner', on=['Dt'])
# sufficient_hourly = pd.merge(sufficient_hourly, weather, how='inner', on=['Dt'])
# # CustIDs || Date | Consumption |||| Weather_variables 

In [4]:
tidy_use = pd.read_pickle(location+'fcast_daily.pkl.zip')

In [9]:
tidy_use_1617 = tidy_use[tidy_use['Dt'] < '2017-10-01']
tidy_use_1718 = tidy_use[tidy_use['Dt'] >= '2017-10-01']

In [12]:
print(tidy_use['DMETERNO'].nunique(), tidy_use_1617['DMETERNO'].nunique(), tidy_use_1718['DMETERNO'].nunique())

6953 5629 6953


In [17]:
tidy_use.groupby('DMETERNO').count()['ID'].mean()

538.1662591687042